In [2]:
import pandas as pd
import numpy as np
# Esta biblioteca sirve para poder eliminar los acentos
from unidecode import unidecode

In [59]:
claves = {"2011": "-11-30", "2012":"-09-30", "2013":"-07-31","2014":"-10-31","2015":"-11-30","2016":"-11-30","2017":"-11-30","2018":"-11-30","2019":"-11-30","2022":"-11-30"}
años = ["2012"]

In [65]:
claves["2011"][1:3]

'11'

In [57]:
# Leemos el archivo de Derechohabientes que queremos
for i in años:
    codificacion = 'windows-1252'   #'utf-8'
    archivo = "pda-" + i + claves[i] +".csv"
    Derechohabientes = pd.read_csv("C:/Users/ccamacho/OneDrive - SESNA/DIA/Proyectos Especiales/IMSS/Derechohabientes/Bases_pda/"+archivo, encoding = codificacion)


In [58]:
# Creamos una copia de la tabla que estaba en nuestro archivo
DH = Derechohabientes.copy()
DH.head()

,ID_DELEG_RP|ID_SUBDEL_RP|ID_UMF_RP|NOMBRE_UMF_RP|ST_TIT_FAM|ID_CALIDAD|CVE_GENERO|CVE_RANGO_EDAD|ST_CONSULTORIO|ID_TURNO|ID_CONSULTORIO|TOT_CASOS
0,12|2|26|UMF 026|2|13|2 |E10|1|V|17|38
1,12|2|9|UMF 009|2|2|2 |E14|1|M|22|2
2,12|2|9|UMF 009|2|6|2 |E16|1|M|18|5
3,12|2|9|UMF 009|1|1|1 |E17|0|0|9998|125
4,12|2|9|UMF 009|2|2|2 |E17|1|V|24|23


In [53]:
# ocuparemos esta variable mas adelante
encabezados = DH.columns.tolist() # extraemos el nombre de la coolumna y lo transformamos a una lista para trabajar con ellos
encabezados = ''.join(encabezados) #Este comando lo utilizamos para transformar la lista de encabezados en una cadena
names = pd.DataFrame({"name" : [encabezados]}) # Hacemos un DataFrame con los encabezados
names = names['name'].str.split(pat='|',expand=True) # Separamos los encabezados ya que estaban todos en una misma columna
names = names.iloc[0] # guardamos los encabezados en la misma variable "names" ya separados

In [54]:
# Dividimos toda nuestra base por el separados
if i=="2011" or i=="2012" or i=="2013" or i=="2014" or i == "2015" or i == "2016":
    DH = DH['ID_DELEG_RP|ID_SUBDEL_RP|ID_UMF_RP|NOMBRE_UMF_RP|ST_TIT_FAM|ID_CALIDAD|CVE_GENERO|CVE_RANGO_EDAD|ST_CONSULTORIO|ID_TURNO|ID_CONSULTORIO|TOT_CASOS'].str.split(pat='|',expand=True)
else:
    DH = DH['ID_DELEG_RP|ID_SUBDEL_RP|ID_UMF_RP|NOMBRE_UMF_RP|ST_TIT_FAM|ID_CALIDAD|CVE_SEXO|CVE_RANGO_EDAD|ST_CONSULTORIO|ID_TURNO|ID_CONSULTORIO|TOT_CASOS'].str.split(pat='|',expand=True)

In [55]:
# ahgregamos los encabezados con la variable antes creada 
DH.columns = names
# observamos los primeros 5 registros para verificar que los encabezados sean correctos
DH.head()

,ID_DELEG_RP,ID_SUBDEL_RP,ID_UMF_RP,NOMBRE_UMF_RP,ST_TIT_FAM,ID_CALIDAD,CVE_GENERO,CVE_RANGO_EDAD,ST_CONSULTORIO,ID_TURNO,ID_CONSULTORIO,TOT_CASOS
0,31,7,73,UMF 073,2,14,2,E10,1,M,19,53
1,29,1,67,UMF 067,2,2,2,E19,1,M,15,147
2,29,1,67,UMF 067,2,2,1,E19,1,M,15,42
3,16,1,249,UMF 249,2,13,1,E9,1,M,5,18
4,17,13,82,UMF 082,2,14,1,E10,1,V,2,58


In [265]:
# Observamos la dimensión del DF
DH.shape

(3256584, 12)

In [266]:
#Eliminar ST_CONSULTORIO, ID_TURNO, ID_CONSULTORIO
DH = DH.drop(["ST_CONSULTORIO", "ID_TURNO", "ID_CONSULTORIO"], axis=1)
DH.head()

,ID_DELEG_RP,ID_SUBDEL_RP,ID_UMF_RP,NOMBRE_UMF_RP,ST_TIT_FAM,ID_CALIDAD,CVE_SEXO,CVE_RANGO_EDAD,TOT_CASOS
0,1,1,2,UMF 002,1,1,1,E10,2
1,1,1,2,UMF 002,1,1,1,E10,1
2,1,1,2,UMF 002,1,1,1,E10,2
3,1,1,2,UMF 002,1,1,1,E10,4
4,1,1,2,UMF 002,1,1,1,E10,1


In [267]:
# Transformamos la columna de TOT_CASOS a enteros
DH['TOT_CASOS'] = DH['TOT_CASOS'].astype(int)
# Agrupamos para consguir el número total de derechohabientes y reducir las filas.
if i=="2011" or i=="2012" or i=="2013" or i=="2014" or i == "2015" or i == "2016":
    DH = DH.groupby(["ID_DELEG_RP","ID_SUBDEL_RP","ID_UMF_RP","NOMBRE_UMF_RP","ST_TIT_FAM","ID_CALIDAD","CVE_GENERO","CVE_RANGO_EDAD"]).sum().reset_index()
    DH.shape
else:
    DH = DH.groupby(["ID_DELEG_RP","ID_SUBDEL_RP","ID_UMF_RP","NOMBRE_UMF_RP","ST_TIT_FAM","ID_CALIDAD","CVE_SEXO","CVE_RANGO_EDAD"]).sum().reset_index()
    DH.shape

In [268]:
DH['TOT_CASOS'].sum() #Verificamos que la cifra total de derechohabientes sea congruente.

61371766

In [269]:
# Creamos una columna llamada "CLAVE" para identificar la institución
DH['Clave_dh'] = DH["ID_DELEG_RP"] + "ID_DEL" + DH["ID_SUBDEL_RP"] + "ID_SUBDEL" + DH["ID_UMF_RP"] + "ID_UMF"
#Vamos a mover la columna que acabamos de crear a una posición más adecuada
clave = DH.pop('Clave_dh')
DH.insert(3,'Clave_dh',clave) 
DH.head()

,ID_DELEG_RP,ID_SUBDEL_RP,ID_UMF_RP,Clave_dh,NOMBRE_UMF_RP,ST_TIT_FAM,ID_CALIDAD,CVE_SEXO,CVE_RANGO_EDAD,TOT_CASOS
0,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E1,2
1,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E10,365
2,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E11,3774
3,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E12,3309
4,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E13,3025


In [270]:
#Con este linea podemos saber la cantidad que hay para la columna "CLAVE"
len(DH['Clave_dh'].unique())

1278

In [14]:
# Ahora vamos a leer el archivo, donde esta el catálogo de cada una de las instituciones
catalogo = pd.read_excel("catalogo_UMF.xlsx", sheet_name="Catálogo")

In [271]:
# Creamos una copia con la variables siguiente
cat = catalogo.copy()
cat.head(2)

,ID_DELEG_RP,ID_SUBDEL_RP,ID_UMF_RP,DELEGACIÓN,SUBDELEGACIÓN,UNIDAD,ID_ENT,ID_MUN,ENTIDAD,MUNICIPIO,ID_LOCAL,LOCALIDAD
0,1,1.0,2.0,Aguascalientes,Aguascalientes Norte,UMF 2 Rincón de Romos,1.0,7.0,Aguascalientes,Rincón de Romos,1.0,Rincón de Romos
1,1,1.0,3.0,Aguascalientes,Aguascalientes Norte,UMF 3 Pabellón Arteaga,1.0,6.0,Aguascalientes,Pabellón de Arteaga,1.0,Pabellón de Arteaga


In [272]:
# transformamos el tipo de dato de cada columnas para poder trabajarlo
cat["ID_SUBDEL_RP"] = cat["ID_SUBDEL_RP"].astype("Int64")
cat["ID_UMF_RP"] = cat["ID_UMF_RP"].astype("Int64")

# Ahora transformamos todo, a un tipo de dato str
cat["ID_DELEG_RP"] = cat["ID_DELEG_RP"].astype(str)
cat["ID_SUBDEL_RP"] = cat["ID_SUBDEL_RP"].astype(str)
cat["ID_UMF_RP"] = cat["ID_UMF_RP"].astype(str)

#Mostramos los primero cinco registros del DF
cat.head()

,ID_DELEG_RP,ID_SUBDEL_RP,ID_UMF_RP,DELEGACIÓN,SUBDELEGACIÓN,UNIDAD,ID_ENT,ID_MUN,ENTIDAD,MUNICIPIO,ID_LOCAL,LOCALIDAD
0,1,1,2,Aguascalientes,Aguascalientes Norte,UMF 2 Rincón de Romos,1.0,7.0,Aguascalientes,Rincón de Romos,1.0,Rincón de Romos
1,1,1,3,Aguascalientes,Aguascalientes Norte,UMF 3 Pabellón Arteaga,1.0,6.0,Aguascalientes,Pabellón de Arteaga,1.0,Pabellón de Arteaga
2,1,1,5,Aguascalientes,Aguascalientes Norte,UMF 5 Asientos,1.0,2.0,Aguascalientes,Asientos,1.0,Asientos
3,1,1,6,Aguascalientes,Aguascalientes Norte,UMF 6 Jesús María,1.0,5.0,Aguascalientes,Jesús María,1.0,Jesús María
4,1,1,7,Aguascalientes,Aguascalientes Norte,UMF 7 San Marcos,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes


In [273]:
# Creamos la clave con la que vamos a conectar la base de derechohabientes DH
cat["Clave_Cta"] = cat["ID_DELEG_RP"] + "ID_DEL" + cat["ID_SUBDEL_RP"] + "ID_SUBDEL" + cat["ID_UMF_RP"] +"ID_UMF"
#Vamos a mover la columna que acabamos de crear a una posición más adecuada
clave_cta = cat.pop('Clave_Cta')
cat.insert(3,'Clave_Cta',clave_cta) 
cat.head()

,ID_DELEG_RP,ID_SUBDEL_RP,ID_UMF_RP,Clave_Cta,DELEGACIÓN,SUBDELEGACIÓN,UNIDAD,ID_ENT,ID_MUN,ENTIDAD,MUNICIPIO,ID_LOCAL,LOCALIDAD
0,1,1,2,1ID_DEL1ID_SUBDEL2ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 2 Rincón de Romos,1.0,7.0,Aguascalientes,Rincón de Romos,1.0,Rincón de Romos
1,1,1,3,1ID_DEL1ID_SUBDEL3ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 3 Pabellón Arteaga,1.0,6.0,Aguascalientes,Pabellón de Arteaga,1.0,Pabellón de Arteaga
2,1,1,5,1ID_DEL1ID_SUBDEL5ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 5 Asientos,1.0,2.0,Aguascalientes,Asientos,1.0,Asientos
3,1,1,6,1ID_DEL1ID_SUBDEL6ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 6 Jesús María,1.0,5.0,Aguascalientes,Jesús María,1.0,Jesús María
4,1,1,7,1ID_DEL1ID_SUBDEL7ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 7 San Marcos,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes


In [274]:
# Unimos las bases DH que es la base derechohabientes, con la base catálogos
DH_1 = pd.merge(DH, cat , left_on='Clave_dh',right_on='Clave_Cta', how='left')
DH_1.head()

,ID_DELEG_RP_x,ID_SUBDEL_RP_x,ID_UMF_RP_x,Clave_dh,NOMBRE_UMF_RP,ST_TIT_FAM,ID_CALIDAD,CVE_SEXO,CVE_RANGO_EDAD,TOT_CASOS,...,Clave_Cta,DELEGACIÓN,SUBDELEGACIÓN,UNIDAD,ID_ENT,ID_MUN,ENTIDAD,MUNICIPIO,ID_LOCAL,LOCALIDAD
0,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E1,2,...,1ID_DEL1ID_SUBDEL10ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 10\nAguascalientes,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes
1,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E10,365,...,1ID_DEL1ID_SUBDEL10ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 10\nAguascalientes,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes
2,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E11,3774,...,1ID_DEL1ID_SUBDEL10ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 10\nAguascalientes,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes
3,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E12,3309,...,1ID_DEL1ID_SUBDEL10ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 10\nAguascalientes,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes
4,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E13,3025,...,1ID_DEL1ID_SUBDEL10ID_UMF,Aguascalientes,Aguascalientes Norte,UMF 10\nAguascalientes,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes


In [275]:
# Observamos la dimensión de nuestrp DF
DH_1.shape

(391477, 23)

In [276]:
# las nombres de las columnas nos ayudan a saber cuales vamos a borrar
DH_1.columns

Index(['ID_DELEG_RP_x', 'ID_SUBDEL_RP_x', 'ID_UMF_RP_x', 'Clave_dh',
       'NOMBRE_UMF_RP', 'ST_TIT_FAM', 'ID_CALIDAD', 'CVE_SEXO',
       'CVE_RANGO_EDAD', 'TOT_CASOS', 'ID_DELEG_RP_y', 'ID_SUBDEL_RP_y',
       'ID_UMF_RP_y', 'Clave_Cta', 'DELEGACIÓN', 'SUBDELEGACIÓN', 'UNIDAD',
       'ID_ENT', 'ID_MUN', 'ENTIDAD', 'MUNICIPIO', 'ID_LOCAL', 'LOCALIDAD'],
      dtype='object')

In [277]:
# Eliminamos las columnas que no necesitamos
#del DH_1['DELEGACIÓN']
del DH_1['ID_DELEG_RP_y']
del DH_1['ID_SUBDEL_RP_y']
del DH_1['ID_UMF_RP_y']
del DH_1['Clave_Cta']
del DH_1['SUBDELEGACIÓN']

In [278]:
#len(DH_1['UNIDAD'].unique())
DH_1['TOT_CASOS'].sum()

61371766

In [279]:
# Creamos un nuevo DataFrame, para agrpuar y asi solo tener las unidades Unicas, un archivo más pequeño
if i=="2011" or i=="2012" or i=="2013" or i=="2014" or i == "2015" or i == "2016":
    grupo = DH_1.copy()
    grupo["Clave"] = grupo["ID_DELEG_RP_x"] + grupo["ID_SUBDEL_RP_x"] + grupo["ID_UMF_RP_x"]
    del grupo["ID_DELEG_RP_x"], grupo["ID_SUBDEL_RP_x"], grupo["ID_UMF_RP_x"],grupo["ST_TIT_FAM"],grupo["ID_CALIDAD"]
    del grupo["CVE_GENERO"], grupo["CVE_RANGO_EDAD"]
else:
    grupo = DH_1.copy()
    grupo["Clave"] = grupo["ID_DELEG_RP_x"] + grupo["ID_SUBDEL_RP_x"] + grupo["ID_UMF_RP_x"]
    del grupo["ID_DELEG_RP_x"], grupo["ID_SUBDEL_RP_x"], grupo["ID_UMF_RP_x"],grupo["ST_TIT_FAM"],grupo["ID_CALIDAD"]
    del grupo["CVE_SEXO"], grupo["CVE_RANGO_EDAD"]

In [280]:
# Con esta línea podemos ver todas las columnas y fila donde tenemos NaN, o datos faltantes
grupo[grupo.isna().any(axis=1)]

,Clave_dh,NOMBRE_UMF_RP,TOT_CASOS,DELEGACIÓN,UNIDAD,ID_ENT,ID_MUN,ENTIDAD,MUNICIPIO,ID_LOCAL,LOCALIDAD,Clave
264613,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441
264614,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441
264615,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441
264616,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441
264617,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441
...,...,...,...,...,...,...,...,...,...,...,...,...
264997,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441
264998,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441
264999,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441
265000,29ID_DEL4ID_SUBDEL41ID_UMF,UMF 041,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29441


In [281]:
# Rellenamos todos los espacion sin dato o mejor dicho que no empataron con "Sin información"
grupo = grupo.fillna('Sin informacion')

In [282]:
# Agrupamos para poder tener por unidad medica los Derechohabientes
#grupo = grupo.groupby(["Clave_dh","Clave","NOMBRE_UMF_RP","UNIDAD","DELEGACIÓN","ID_ENT","ENTIDAD","ID_MUN","MUNICIPIO",
#                      "ID_LOCAL","LOCALIDAD"]).sum().reset_index()
grupo = grupo.groupby(["Clave_dh","Clave","UNIDAD","DELEGACIÓN","ID_ENT","ENTIDAD","ID_MUN","MUNICIPIO",
                      "ID_LOCAL","LOCALIDAD"]).sum().reset_index()

C:\Users\ccamacho\AppData\Local\Temp\ipykernel_12476\125273881.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  "ID_LOCAL","LOCALIDAD"]).sum().reset_index()


In [283]:
# Observamos las dimensiones de nuestro DF
grupo.shape

(1278, 11)

In [284]:
#Corroboramos que sigan siendo la misma cantidad de Derechohabientes
grupo['TOT_CASOS'].sum()

61371766

In [55]:
#Agregamos columna con el nombre de la unidad, para no modificar la original
#Esta['Nombre_modificado'] = Esta['NOMBRE DE LA UNIDAD']
#grupo['Nombre_mod'] = grupo['UNIDAD']

In [56]:
# # Vamos a eliminar cierto caracteres, para hacer las columnas lo mas similares posibles
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].str.lower()
# grupo['Nombre_mod'] = grupo['Nombre_mod'].str.lower()
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace('\s+', '', regex=True) # Eliminamos espacios
# grupo['Nombre_mod'] = grupo['Nombre_mod'].replace('\s+', '', regex=True) # Eliminamos espacios
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace('\.', '', regex=True) # Eliminamos puntos
# grupo['Nombre_mod'] = grupo['Nombre_mod'].replace('\.', '', regex=True) #Eliminamos puntos
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace(',', '', regex=True) # Eliminamos comas
# grupo['Nombre_mod'] = grupo['Nombre_mod'].replace(',', '', regex=True) # Eliminamos comas
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace('-', '', regex=True) # Eliminamos guíon
# grupo['Nombre_mod'] = grupo['Nombre_mod'].replace('-', '', regex=True) # Eliminamos guíon
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace('_', '', regex=True) # Eliminamos guion bajo
# grupo['Nombre_mod'] = grupo['Nombre_mod'].replace('_', '', regex=True) # Eliminamos guion bajo
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].apply(unidecode) # Eliminamos acentos
# grupo['Nombre_mod'] = grupo['Nombre_mod'].apply(unidecode)# Eliminamos acentos

In [81]:
#catalogo_D = pd.read_excel("Derechohabientes.xlsx",usecols="A,E,N") # Leemos el catálogo
catalogo_D = pd.read_excel("catálogo_derechohabientes.xlsx",usecols="A,C,k") # Leemos el catálogo
#catalogo_D = catalogo_D[catalogo_D["Año"] == int(i)] # Filtramos por el año que estamos trabajando

In [285]:
# Unimos el DF de grupo con el de estableciemintos para poder encontrarle el CLUE a cada unidad medica
catalogo_b = catalogo_D.copy()
clues = pd.merge(grupo, catalogo_b, left_on='Clave_dh',right_on='Clave_dh', how='left') 
del clues["UNIDAD_y"]
clues.head(2)

,Clave_dh,Clave,UNIDAD_x,DELEGACIÓN,ID_ENT,ENTIDAD,ID_MUN,MUNICIPIO,ID_LOCAL,LOCALIDAD,TOT_CASOS,CLUES
0,10ID_DEL13ID_SUBDEL10ID_UMF,101310,UMF 10 Gómez Palacio,Durango,10.0,Durango,7.0,Gómez Palacio,1.0,Gómez Palacio,106902,DGIMS000100
1,10ID_DEL13ID_SUBDEL11ID_UMF,101311,UMF 11 Cd. Lerdo,Durango,10.0,Durango,12.0,Lerdo,1.0,Ciudad Lerdo,69490,DGIMS000182


In [286]:
# Observamos la dimensión de nuestro nuevo DF
clues.shape

(1278, 12)

In [287]:
# Observamos la cantidad de datos unicos que tenemos en CLUE
# lo que siginfica que es el número que coicideron
len(clues['CLUES'].unique())

1278

In [93]:
# Esta nueva base se ocupara para poder traer las variables que nos hacen falta como los "CLUES", "Lat", "Lon"
establecimientos = pd.read_excel('C:/Users/ccamacho/OneDrive - SESNA/DIA/Proyectos Especiales/IMSS/Bases_Quejas/ESTABLECIMIENTO_SALUD_202308.xlsx', usecols="A,E,H,I,L,M,AQ,AT,BA,BB")

In [288]:
# Creamos una copia
Esta = establecimientos.copy()
Esta.head(2)

,CLUES,JURISDICCION,CLAVE DE TIPOLOGIA,NOMBRE DE TIPOLOGIA,NOMBRE DE LA UNIDAD,NOMBRE COMERCIAL,CLAVE DE LA INS ADM,NIVEL ATENCION,LATITUD,LONGITUD
0,ASDIF000011,AGUASCALIENTES,99,NO ESPECIFICADO,DIF AGUASCALIENTES,DIF AGUASCALIENTES,NES,NO APLICA,21.8677,-102.309
1,ASDIF000023,RINCÓN DE ROMOS,99,NO ESPECIFICADO,DIF COSÍO,DIF COSÍO,NES,NO APLICA,22.3399,-102.264


In [289]:
clues = pd.merge(clues, Esta, left_on='CLUES',right_on='CLUES', how='left') 
clues = clues.rename(columns={"UNIDAD_x":"UNIDAD"})
clues.head(2)

,Clave_dh,Clave,UNIDAD,DELEGACIÓN,ID_ENT,ENTIDAD,ID_MUN,MUNICIPIO,ID_LOCAL,LOCALIDAD,...,CLUES,JURISDICCION,CLAVE DE TIPOLOGIA,NOMBRE DE TIPOLOGIA,NOMBRE DE LA UNIDAD,NOMBRE COMERCIAL,CLAVE DE LA INS ADM,NIVEL ATENCION,LATITUD,LONGITUD
0,10ID_DEL13ID_SUBDEL10ID_UMF,101310,UMF 10 Gómez Palacio,Durango,10.0,Durango,7.0,Gómez Palacio,1.0,Gómez Palacio,...,DGIMS000100,GÓMEZ PALACIO,UMF,UNIDAD DE MEDICINA FAMILIAR,UMF 10 GOMEZ PALACIO,UMF 10 GOMEZ PALACIO,IMSS,PRIMER NIVEL,25.5562,-103.506
1,10ID_DEL13ID_SUBDEL11ID_UMF,101311,UMF 11 Cd. Lerdo,Durango,10.0,Durango,12.0,Lerdo,1.0,Ciudad Lerdo,...,DGIMS000182,GÓMEZ PALACIO,UMF,UNIDAD DE MEDICINA FAMILIAR,UMF 11 CD. LERDO,UMF 11 CD. LERDO,IMSS,PRIMER NIVEL,25.5415,-103.522


In [290]:
clues.columns

Index(['Clave_dh', 'Clave', 'UNIDAD', 'DELEGACIÓN', 'ID_ENT', 'ENTIDAD',
       'ID_MUN', 'MUNICIPIO', 'ID_LOCAL', 'LOCALIDAD', 'TOT_CASOS', 'CLUES',
       'JURISDICCION', 'CLAVE DE TIPOLOGIA', 'NOMBRE DE TIPOLOGIA',
       'NOMBRE DE LA UNIDAD', 'NOMBRE COMERCIAL', 'CLAVE DE LA INS ADM',
       'NIVEL ATENCION', 'LATITUD', 'LONGITUD'],
      dtype='object')

In [291]:
# Eliminamos duplicados
#clues = clues.drop_duplicates(subset=['Clave_dh','Clave','NOMBRE_UMF_RP','UNIDAD','DELEGACIÓN','ID_ENT','ENTIDAD',
#                                      'ID_MUN','MUNICIPIO','ID_LOCAL','LOCALIDAD','TOT_CASOS','Nombre_mod'])
clues = clues.drop_duplicates(subset=['Clave_dh','Clave','UNIDAD','DELEGACIÓN','ID_ENT','ENTIDAD',
                                      'ID_MUN','MUNICIPIO','ID_LOCAL','LOCALIDAD','TOT_CASOS'])

In [292]:
# No aseguramos que no tengamos filas vacias o que no haya datos faltantes
# En este caso si tenemos datos faltantes 
clues['CLUES'].isnull().any()

False

In [62]:
#catalogo_D = pd.read_excel("Derechohabientes.xlsx",usecols="A,E,N") # Leemos el catálogo
#catalogo_D = pd.read_excel("catálogo_derechohabientes.xlsx",usecols="A,C,k") # Leemos el catálogo
#catalogo_D = catalogo_D[catalogo_D["Año"] == int(i)] # Filtramos por el año que estamos trabajando

In [66]:
###############################################################################################################################################################################################
############################################## Rellenamos lo CLUES pendientes con un catálogo que se elaboro antes ############################################################################
###############################################################################################################################################################################################

# #pd.merge(grupo, Esta  , left_on='Nombre_mod',right_on='Nombre_modificado', how='left')
# catalogo_b = catalogo_D.copy() # Creamo una copia

# filtro=clues['CLUES'].isnull() # creamos el filtro con los nulos
# clues_filtro = clues[filtro] #guardamos solo los nulos
# clues_nulos = pd.merge(grupo, catalogo_b,  on="Clave_dh") # Agregamos los CLUES a los que no tengan
# del clues_nulos["CLUES_x"] # Eliminamos la columna CLUES que contenia a los nulos
# # del clues_nulos["Año"] # Eliminamos la columna "Año"
# del clues_nulos["JURISDICCION"],clues_nulos["CLAVE DE TIPOLOGIA"],clues_nulos["NOMBRE DE TIPOLOGIA"],clues_nulos["NOMBRE DE LA UNIDAD"],clues_nulos["NOMBRE COMERCIAL"] # Eliminamos columnas vacias
# del clues_nulos["CLAVE DE LA INS ADM"],clues_nulos["NIVEL ATENCION"],clues_nulos["LATITUD"],clues_nulos["LONGITUD"],clues_nulos["Nombre_modificado"] # Eliminamos columnas vacias
# clues_nulos = clues_nulos.rename(columns={"CLUES_y":"CLUES"}) #Cambiamos el nombre de la columna CLUES_y por CLUES ya que esta ya no contiene datos nulos
# clues_nulos = pd.merge(clues_nulos, Esta, left_on='CLUES',right_on='CLUES', how='left') # Unimos los clues que eran nulos con el la base de "establecimientos" para traer la información que necesitamos

# # Eliminamos de nuestra tabla origianl los nulos, para quedarnos solo con los que si tenian dato
# clues = clues.drop(clues_filtro.index)
# clues = pd.concat([clues,clues_nulos],axis=0) # Concatenamos los nulos que ahora ya tienen dato con la tabla original pero solo los registros que si tenian dato

In [293]:
# Nos aseguramos que la cantidad de Derechohabientes siga siendo la misma
clues['TOT_CASOS'].sum()

61371766

In [294]:
clues_duplicados = clues[clues.duplicated(subset=["CLUES"],keep=False)]
clues_duplicados

,Clave_dh,Clave,UNIDAD,DELEGACIÓN,ID_ENT,ENTIDAD,ID_MUN,MUNICIPIO,ID_LOCAL,LOCALIDAD,...,CLUES,JURISDICCION,CLAVE DE TIPOLOGIA,NOMBRE DE TIPOLOGIA,NOMBRE DE LA UNIDAD,NOMBRE COMERCIAL,CLAVE DE LA INS ADM,NIVEL ATENCION,LATITUD,LONGITUD


In [295]:
# Podemos usar esta línea para tranformar nuestro DF a un archivo excel.
#clues.to_excel("Derechohabientes_" + i +".xlsx",index=0)

En este punto hay que trabajar los CLUES en el archivo Excel, pegar los que faltan del archivo de derechohabientes

## Aqui comenzaremos a trabajar con la base mas grande

In [150]:
# Esta['Nombre_modificado'] = Esta['NOMBRE DE LA UNIDAD'] #Agregamos columna con el nombre de la unidad, para no modificar la original
# DH_1['Nombre_mod'] = DH_1['UNIDAD'] #Agregamos columna con el nombre de la unidad, para no modificar la original

In [151]:
#cat["ID_UMF_RP"] = cat["ID_UMF_RP"].astype("Int64")
# Ahora transformamos todo, a un tipo de dato str
#cat["ID_DELEG_RP"] = cat["ID_DELEG_RP"].astype(str)

In [152]:
# Vamos a eliminar cierto caracteres, para hacer las columnas lo mas similares posibles
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].str.lower()
# DH_1['Nombre_mod'] = DH_1['Nombre_mod'].str.lower()
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace('\s+', '', regex=True) # Eliminamos espacios
# DH_1['Nombre_mod'] = DH_1['Nombre_mod'].replace('\s+', '', regex=True) # Eliminamos espacios
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace('\.', '', regex=True) # Eliminamos puntos
# DH_1['Nombre_mod'] = DH_1['Nombre_mod'].replace('\.', '', regex=True) #Eliminamos puntos
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace(',', '', regex=True) # Eliminamos comas
# DH_1['Nombre_mod'] = DH_1['Nombre_mod'].replace(',', '', regex=True) # Eliminamos comas
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace('-', '', regex=True) # Eliminamos guíon
# DH_1['Nombre_mod'] = DH_1['Nombre_mod'].replace('-', '', regex=True) # Eliminamos guíon
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].replace('_', '', regex=True) # Eliminamos guion bajo
# DH_1['Nombre_mod'] = DH_1['Nombre_mod'].replace('_', '', regex=True) # Eliminamos guion bajo
# Esta['Nombre_modificado'] = Esta['Nombre_modificado'].astype(str).apply(unidecode) # Eliminamos acentos
# DH_1['Nombre_mod'] = DH_1['Nombre_mod'].astype(str).apply(unidecode)# Eliminamos acentos

In [153]:
# Mismo procedmiento que el anterior unimos con los datos del archivo establecimientos
# clues_2 = pd.merge(DH_1, Esta, left_on='Nombre_mod',right_on='Nombre_modificado', how='left')
# clues_2.head(2)

,ID_DELEG_RP_x,ID_SUBDEL_RP_x,ID_UMF_RP_x,Clave_dh,NOMBRE_UMF_RP,ST_TIT_FAM,ID_CALIDAD,CVE_GENERO,CVE_RANGO_EDAD,TOT_CASOS,...,JURISDICCION,CLAVE DE TIPOLOGIA,NOMBRE DE TIPOLOGIA,NOMBRE DE LA UNIDAD,NOMBRE COMERCIAL,CLAVE DE LA INS ADM,NIVEL ATENCION,LATITUD,LONGITUD,Nombre_modificado
0,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E0,693,...,AGUASCALIENTES,UMF,UNIDAD DE MEDICINA FAMILIAR,UMF 10 AGUASCALIENTES,UMF 10 AGUASCALIENTES,IMSS,PRIMER NIVEL,21.9002,-102.293,umf10aguascalientes
1,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E1,134,...,AGUASCALIENTES,UMF,UNIDAD DE MEDICINA FAMILIAR,UMF 10 AGUASCALIENTES,UMF 10 AGUASCALIENTES,IMSS,PRIMER NIVEL,21.9002,-102.293,umf10aguascalientes


In [297]:
clues_2 = pd.merge(DH_1, catalogo_b, left_on='Clave_dh',right_on='Clave_dh', how='left') 
clues_2.head(2)

,ID_DELEG_RP_x,ID_SUBDEL_RP_x,ID_UMF_RP_x,Clave_dh,NOMBRE_UMF_RP,ST_TIT_FAM,ID_CALIDAD,CVE_SEXO,CVE_RANGO_EDAD,TOT_CASOS,DELEGACIÓN,UNIDAD_x,ID_ENT,ID_MUN,ENTIDAD,MUNICIPIO,ID_LOCAL,LOCALIDAD,UNIDAD_y,CLUES
0,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E1,2,Aguascalientes,UMF 10\nAguascalientes,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes,UMF 10\nAguascalientes,ASIMS000045
1,1,1,10,1ID_DEL1ID_SUBDEL10ID_UMF,UMF 010,1,1,1,E10,365,Aguascalientes,UMF 10\nAguascalientes,1.0,1.0,Aguascalientes,Aguascalientes,1.0,Aguascalientes,UMF 10\nAguascalientes,ASIMS000045


In [298]:
# Dimensión con la base completa
clues_2.shape

(391477, 20)

In [299]:
len(clues_2['CLUES'].unique())

1278

Hasta este punto el código esta correcto, nos faltan CLUES por agregar, pero se ve a crear solo un DF con las unidades 
que hacen falta

In [300]:
clues_expe = clues_2.copy()

In [301]:
clues_expe.columns

Index(['ID_DELEG_RP_x', 'ID_SUBDEL_RP_x', 'ID_UMF_RP_x', 'Clave_dh',
       'NOMBRE_UMF_RP', 'ST_TIT_FAM', 'ID_CALIDAD', 'CVE_SEXO',
       'CVE_RANGO_EDAD', 'TOT_CASOS', 'DELEGACIÓN', 'UNIDAD_x', 'ID_ENT',
       'ID_MUN', 'ENTIDAD', 'MUNICIPIO', 'ID_LOCAL', 'LOCALIDAD', 'UNIDAD_y',
       'CLUES'],
      dtype='object')

In [302]:
clues_expe['CLUES'].isnull().any()

False

In [307]:
clues_expe['TOT_CASOS'].sum()

61371766

In [308]:
del clues_expe["UNIDAD_y"]
clues_expe = clues_expe.rename(columns={"UNIDAD_x":"UNIDAD"})

In [309]:
if i == "2011" or i == "2016" or i == "2015": 
    clues_expe = clues_expe.drop_duplicates(subset=['ID_DELEG_RP_x','ID_SUBDEL_RP_x','ID_UMF_RP_x','Clave_dh','NOMBRE_UMF_RP',
                                                    'ST_TIT_FAM','ID_CALIDAD','CVE_GENERO','CVE_RANGO_EDAD','TOT_CASOS',
                                                    'DELEGACIÓN','UNIDAD','ID_ENT','ENTIDAD','ID_MUN','MUNICIPIO','ID_LOCAL','LOCALIDAD',
                                                    ])
else:
    clues_expe = clues_expe.drop_duplicates(subset=['ID_DELEG_RP_x','ID_SUBDEL_RP_x','ID_UMF_RP_x','Clave_dh','NOMBRE_UMF_RP',
                                                    'ST_TIT_FAM','ID_CALIDAD','CVE_SEXO','CVE_RANGO_EDAD','TOT_CASOS',
                                                    'DELEGACIÓN','UNIDAD','ID_ENT','ENTIDAD','ID_MUN','MUNICIPIO','ID_LOCAL','LOCALIDAD',
                                                    ])

In [310]:
clues_expe['TOT_CASOS'].sum()

61371766

In [311]:
clues_expe.shape

(391477, 19)

In [163]:
###############################################################################################################################################################################################
############################################## Rellenamos lo CLUES pendientes con un catálogo que se elaboro antes ############################################################################
###############################################################################################################################################################################################

#pd.merge(grupo, Esta  , left_on='Nombre_mod',right_on='Nombre_modificado', how='left')
# catalogo_b = catalogo_D.copy() # Creamo una copia

# filtro_c=clues_expe['CLUES'].isnull() # creamos el filtro con los nulos
# clues_filtro_c = clues_expe[filtro_c] #guardamos solo los nulos
# clues_nulos_c = pd.merge(clues_filtro_c, catalogo_b,  on="UNIDAD") # Agregamos los CLUES a los que no tengan
# del clues_nulos_c["CLUES_x"] # Eliminamos la columna CLUES que contenia a los nulos
# del clues_nulos_c["Año"] # Eliminamos la columna "Año"
# del clues_nulos_c["JURISDICCION"],clues_nulos_c["CLAVE DE TIPOLOGIA"],clues_nulos_c["NOMBRE DE TIPOLOGIA"],clues_nulos_c["NOMBRE DE LA UNIDAD"],clues_nulos_c["NOMBRE COMERCIAL"] # Eliminamos columnas vacias
# del clues_nulos_c["CLAVE DE LA INS ADM"],clues_nulos_c["NIVEL ATENCION"],clues_nulos_c["LATITUD"],clues_nulos_c["LONGITUD"],clues_nulos_c["Nombre_modificado"] # Eliminamos columnas vacias
# clues_nulos_c = clues_nulos_c.rename(columns={"CLUES_y":"CLUES"}) #Cambiamos el nombre de la columna CLUES_y por CLUES ya que esta ya no contiene datos nulos
# clues_nulos_c = pd.merge(clues_nulos_c, Esta, left_on='CLUES',right_on='CLUES', how='left') # Unimos los clues que eran nulos con el la base de "establecimientos" para traer la información que necesitamos

# # Eliminamos de nuestra tabla origianl los nulos, para quedarnos solo con los que si tenian dato
# clues_expe = clues_expe.drop(clues_filtro_c.index)
# clues_expe = pd.concat([clues_expe,clues_nulos_c],axis=0) # Concatenamos los nulos que ahora ya tienen dato con la tabla original pero solo los registros que si tenian dato

In [312]:
clues_expe['TOT_CASOS'].sum()

61371766

In [165]:
#clues_expe.to_excel("experimento.xlsx") # CON TODOS

In [270]:
with pd.ExcelWriter("Derechohabientes_" + i + "_" + claves[i][1:3] + ".xlsx") as writer:
    clues_expe.to_excel(writer, sheet_name="Completo", index=False)
    clues.to_excel(writer, sheet_name="Unicos", index=False)

# EXTRA

In [3]:
uno_2011 = pd.read_excel("Derechohabientes_2011_11.xlsx", sheet_name="Unicos")
uno_2011["Año"] = "2011"
uno_2012 = pd.read_excel("Derechohabientes_2012_09.xlsx", sheet_name="Unicos")
uno_2012["Año"] = "2012"
uno_2013 = pd.read_excel("Derechohabientes_2013_07.xlsx", sheet_name="Unicos")
uno_2013["Año"] = "2013"
uno_2014 = pd.read_excel("Derechohabientes_2014_10.xlsx", sheet_name="Unicos")
uno_2014["Año"] = "2014"
uno_2015 = pd.read_excel("Derechohabientes_2015_11.xlsx", sheet_name="Unicos")
uno_2015["Año"] = "2015"
uno_2016 = pd.read_excel("Derechohabientes_2016_11.xlsx", sheet_name="Unicos")
uno_2016["Año"] = "2016"
uno_2017 = pd.read_excel("Derechohabientes_2017_11.xlsx", sheet_name="Unicos")
uno_2017["Año"] = "2017"
uno_2018 = pd.read_excel("Derechohabientes_2018_11.xlsx", sheet_name="Unicos")
uno_2018["Año"] = "2018"
uno_2019 = pd.read_excel("Derechohabientes_2019_11.xlsx", sheet_name="Unicos")
uno_2019["Año"] = "2019"
uno_2022 = pd.read_excel("Derechohabientes_2022_11.xlsx", sheet_name="Unicos")
uno_2022["Año"] = "2022"

In [30]:
Derechohabientes_nuevo = pd.concat([uno_2011,uno_2012,uno_2013,uno_2014,uno_2015,uno_2016,uno_2017,uno_2018,uno_2019,uno_2022],axis=0)

#### Para identificar en la base de derechohabientes, cuales Unidades medicas si aparecen en las encuestas de la EnCal.

## Aun falta

In [34]:
Derechohabientes_nuevo = pd.read_excel("Derechohabientes_completo.xlsx")

In [36]:
Derechohabientes_nuevo["concat"] = Derechohabientes_nuevo["Año"].astype(str) + Derechohabientes_nuevo["CLUES"]

In [38]:
encal_cat_unicas = pd.read_excel("C:/Users/ccamacho/OneDrive - SESNA/DIA/Proyectos Especiales/IMSS/Bases_ensat/Catálogo_EnCal_Unidades_Unicas.xlsx", sheet_name="EnCal_todo")
encal_cat_unicas["concat"] = encal_cat_unicas["Año"].astype(str) + encal_cat_unicas["CLUES"]

In [4]:
Derechohabientes_nuevo.to_excel("Derechohabientes_completo.xlsx", index=False, sheet_name="Derechohabientes")